# Data Integration and Knowledge Representation for AI Systems

Nicholas Harley - nicholas.harley@vub.be

Geraint Wiggins - geraint.wiggins@vub.be

14th October 2024

Github repository: <https://github.com/nick-harley/data-integration>

## Introduction

Many practical AI systems depend on the construction of knowledge bases from distributed heterogenous data sources. One example of such a system is the Flanders Make [AI assisted operater workstation](https://www.flandersairesearch.be/en/research/use-cases/ai-assisted-operator). This workstation augments the capabilities of its operator by responding to voice requests for information and providing detailed instruction throughout the assembly process according the operator's experience level. This is made possible through a combination of subsystems including speech processing, augmented reality and movement tracking. Each of the subsystems generates data which is of value in providing further assistance to the operator. In order to fully utilise these distributed, heterogeneous data sources, they need to be appropriately integrated into a single knowledge base which can be accessed in a uniform way.

In this notebook we demonstrate how such an integrated knowledge base can be constructed using the [CHAKRA](#CHAKRA) knowledge representation framework. CHAKRA consists of a general purpose knowledge model which can be used to capture complex sequential, hierarchical phenomena, as well as a [functional specification for a programming interface](https://github.com/nick-harley/chakra-coq) for accesing and manipulating knowledge structures. Using CHAKRA to construct a knowledge base from distributed heterogenous data requires implementation of interfaces for each of the concrete data sources and engineering the coordination of these interfaces. Here we use the Julia programming language, which has an expressive type system and method overloading, the combination of which is exploited to implement interfaces with type-based dynamic dispatch. 

The rest of this notebook is structured as follows: we introduce the specific use case followed by an overview of CHAKRA. We then describe the [specific knowledge base](#Building-an-Integrated-Knowledge-Base-Using-CHAKRA) that was built and [how it can be queried](#Querying-the-Knowledge-Base). We then give some brief [conclusions(#Conclusions), below which is the all the [Julia code](#Code).  This is a stand alone-alone notebook with no Julia package dependencies. However, CHAKRA is a also available as a [Julia package](https://github.com/nick-harley/Chakra). This notebook is a work-in-progress and maybe updated from time to time.

## Use Case: The AI Assisted Operator Workstation

The [AI Assisted Operator](https://www.flandersairesearch.be/en/research/use-cases/ai-assisted-operator) in consists of dual-arm collaborative robot as well as a microphone, RGB camera, and Arkite projection system. These componenets are integrated using the SkiRos platform and can be utilised by the workstation operator via voice commands and a LLM-based chatbot. A typical interaction with the workstation may involve the operator asking for details of the product being assembled, for instructions or demonstrations, or for additional information about the task at hand. Many of these information requests require access to a variety of different data sources associated with the workstation which include historical execution data, assembly instructions, the robot's world model, order information, and RGB sensor data. 

In this notebook, we focus on one particular example of an operator request: "how long does it take to assemble _X_?" In order to answer this question from the operator, the system must first be able to identify 'X' and locate it in the assembly instructions for the product currently being assembled. It must then find all events and interactions in the historical execution database which related to 'X' and use that information to provide the operator with an estimate of the time required to complete the assembly. Here, we construct a hierarchical knowledge base in which the [XML assembly instructions](dwi.xml) for a simple product - a light switch - are integrated with the [CSV historical execution database](STEPS.csv), and show how this knowledge base can be enriched and queried to answer the above question. 

## CHAKRA

In CHAKRA, knowledge is represented as a sequential, hierarchical configuration of entities called _constituents_. Constitutents can be associated with intrinsic data values called _attribtues_ and extrinsic labels called _properties_. Each constituent is uniquely identifiable and is associated with a (possibly empty) sequence of other constituent identifiers, called it's _particles_. The [CHAKRA signature](https://github.com/nick-harley/chakra-coq) consists of a set of types and operations which together specify a simple programming interface for this general purpose data model. [The CHAKRA Julia module](#chakrajl) specifies this interface as abstract Julia types and functions. A CHAKRA knowledge base is a Julia module which encapsulates some concrete data and implements the CHAKRA interface. Implementing the CHAKRA interface requires defining concrete subtypes of the abstract Julia types, as well as overloading the CHAKRA methods for operating on those types. Knowledge base modules can be augmented and integrated by nesting modules. This requires creating a parent module which imports one or more knowledge base modules and may define new constituents. The interfaces of parent knowledge bases are constructed by defining methods which may call methods from its child knowledge bases.

The [CHAKRA query languauge](#query-language) is higher-level language for accessing information from CHAKRA knowledge bases. This query language is defined in terms of the CHAKRA signature. In Julia, query expressions are constructed as julia objects whose structure correspond to the abstract syntax tree. These objects are also callable on knowledge bases: running a query object with a knowledge base  uses only the interface of that knowledge base. Query objects can be [serialised as JSON](query.json) which can be used as primitive interface between Julia knowledge modules and other programming environments such as Python and SkiRos.

## Building an Integrated Knowledge Base Using CHAKRA

The CHAKRA knowledge base in this notebook integrates the [digital work instructions](dwi.xml) in XML format and the [historical execution data](STEPS.csv) in SQL/csv format. The knowledge base consists of 9 modules, illustrated in the figure below, with the hierarchical configuration of the constituents shown above. Vertical arrangment of modules in the diagram corresponds with the parent-child relation between modules. [DWI](#DWI) encapsulates the assembly instructions. The instructions are represented as a constituent whose particles are the sequence of assembly steps. [Events](#Events) encapsultes the workstation events. They are represented as a flat sequence of constituents whose attributes correspond to the SQL columns. [Seqs](#Seqs) augments the [Events](#Events) knowledge module with constituents which represent complete assmbly sequences. [Chunks](#Chunks) augments the [Events](#Events) knowledge module with constituents which represent individual assembly steps. [ChunkSeqs](#ChunkSeqs) augments the [Chunks](#Chunks) knowledge module with constituents that represent seqeunces of assembly steps. [SeqEx](#SeqEx) and [ChunkEx](#ChunkEx) link the [DWI](#DWI) knowledge module to the [Seqs](#Seqs) and [Chunks](#Chunks) knowledge modules, respectively. These modules define constituents which link the instructions which specific instances of assembly steps and sequences. [KB](#KB) integrates all of above knowledge modules into a single federated knowledge base with an interface which dispatches to the relevant child knowledge module using typing.

<img src="kb.png" width="500">

## Querying the Knowledge Base

For this note book we assume the identification of the required work instruction identifier has already been resolved. In order to answer the question "_How long does it take to assemble the lightswitch?_" information relating to the timing of past assembly sequences must be extracted from the knowledge base and used to calculate a useful estimate of the user. This is done by traversing the hierarchy from high-level constituents and extracting attribute data. The query that corresponds to this request might be written as follows:

```
SELECT( TYPE("isExecutionOf") && LASTPART("DWI.Workflow.Id()")):
    MAP( PROJ(TYPE("Arkite Event"))) ; MAP(GET(":STEP_DURATION")) ; SUM ) ; 
    AVG()
```

This query first selects all constituents in the knowledge base which represent assembly executions and whose last particle is the identifier of the constituent representing the light switch assembly instructions, ``"DWI.Workflow.Id()"``. It then maps over this list of constituents, first performing a hierarchical down-projection on to the type ``"Arkite Event"``. It then extracts all the event durations from those event sequences, and sums them. The resulting list of assebly sequence total durations is then averaged, and this information can be relayed to the operator.

## Conclusions 

This notebook provides a basic demonstration and proof-of-concept regarding the application of the CHAKRA knowledge representation framework in the construction of rich, multiple-hierarchical knowledge bases from distributed, heterogenous data. Two things which are import to highlight by way of conclusion are as follows: First, CHAKRA queries are entirely encoding independent. That is, they do not depend on the format used to encode the data. Users of CHAKRA knowledge structures need only know the vocabulary of attributes and properties used in order to start querying and generally programming with those structures. This is a vitally important feature of CHAKRA: the ability hide irrelavent data encoding details behind the interface and exose only that structure which is relevant for conveying knowledge. Second, while the implementation of programming interfaces is additional work for the authors of data and knowledge, the ammount of code required in Julia is relativelty small in most cases, and the benefits are potentially large from the perspective of interoperability, reusability and reproducibility.

# Code

<a id='chakrajl'></a>
## CHAKRA

In [5]:
module CHAKRA

export option, none, fnd, geta, getp, pts, dom, cts

struct None end
none = None()
option{A} = Union{A,None}

# Abstract Types

abstract type Id end
abstract type Constituent end
abstract type Hierarchy end
abstract type Attribute{N,T} end
abstract type Property{N,T} end

# Registered Attributes and Properties 

__attributes__(::Val{N}) where N = error("No attribute named $N.")
__attributes__(N::Symbol) = __attributes__(Val{N}())
__properties__(::Val{N}) where N = error("No property named $N.")
__properties__(N::Symbol) = __properties__(Val{N}())

# Interface Operations

fnd(x::Id,h::Hierarchy) = none
fnd(x::Id,m::Module) = fnd(x,m.__data__)

geta(::Attribute,c::Constituent) = none
geta(N::Symbol,c) = geta(__attributes__(Val{N}()),c)

getp(::Property,c::Constituent) = none
getp(N::Symbol,c) = getp(__properties__(Val{N}()),c)

pts(c::Constituent) = Id[]

dom(h::Hierarchy) = Id[]
dom(m::Module) = dom(m.__data__)

#sequence(ids,kb) = [fnd(x,kb) for x in ids]
#cts(kb) = sequence(dom(kb),kb)

cts(kb) = Pair{Id,Constituent}[x=>fnd(x,kb) for x in dom(kb)]

sequence(ids,kb) = begin
    isempty(ids) && return Constituent[]
    c = fnd(ids[1],kb)
    c isa None && return none
    r = sequence(ids[2:end],kb)
    r isa None && return none
    return Constituent[c,r...]
end

# PROPERTIES

export DESCRIPTION, TYPE

struct DESCRIPTION <: Property{:DESCRIPTION,String} end
__properties__(::Val{:DESCRIPTION}) = DESCRIPTION()

struct TYPE <: Property{:TYPE,String} end
__properties__(::Val{:TYPE}) = TYPE()

end

using Main.CHAKRA

<a id='dwi'></a>
## DWI

In [7]:
module DWI

using Main.CHAKRA
using LightXML, JSON

# Concrete Data

xml = parse_file("dwi.xml")
r = root(xml)
wfspec = r["WorkflowSpecification"][1]
wfdescription = content(r["Description"][1])
wfnodes = wfspec["WorkflowSpecificationNode"][2:7]
wfnodetypes = [content(n["WorkflowSpecificationNodeTypeID"][1]) for n in wfnodes]
properties = [n["WorkflowSpecificationNodeProperty"] for n in wfnodes]
names = [content(p[1]["Value"][1]["ValueString"][1]) for p in properties]
descriptions = [JSON.parse(content(p[4]["Value"][1]["ValueString"][1])) for p in properties]

abstract type Id <: CHAKRA.Id end

module Steps

using ..CHAKRA
import ..DWI: descriptions, names, wfnodes

struct Id{N} <: Main.DWI.Id end
struct Step{N} <: CHAKRA.Constituent 
    Step{N}() where N = N > length(wfnodes) ? none : new{N}()
end
struct Data <: CHAKRA.Hierarchy end

struct STEP_NAME <: CHAKRA.Property{:STEP_NAME,String} end
CHAKRA.__properties__(::Val{:STEP_NAME}) = STEP_NAME()

CHAKRA.fnd(::Id{N},::Data) where N = Step{N}()
CHAKRA.pts(::Step) = CHAKRA.Id[]
CHAKRA.getp(::TYPE,::Step) = "Assembly Step"
CHAKRA.getp(::DESCRIPTION,c::Step{N}) where N = descriptions[N]
CHAKRA.getp(::STEP_NAME,c::Step{N}) where N = names[N]
CHAKRA.dom(::Data) = [Id{N}() for N in 1:6]

__data__ = Data()

end

module Workflow

using ..CHAKRA
using ..Steps
import ..DWI: wfdescription

struct Id <: Main.DWI.Id end
struct WF <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(::Id,::Data) = WF()
CHAKRA.pts(::WF) = [Steps.Id{N}() for N in 1:6]
CHAKRA.getp(::TYPE,::WF) = "Workflow Specification"
CHAKRA.getp(::DESCRIPTION,c::WF) = wfdescription
CHAKRA.dom(::Data) = [Id()]

__data__ = Data()

end

struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(x::ID,::Data) where {ID<:Id} = fnd(x,parentmodule(ID))
CHAKRA.dom(::Data) = [dom(Workflow)...,dom(Steps)...]

__data__ = Data()

end

Main.DWI

In [8]:
using Main.DWI

In [9]:
dom(DWI)

7-element Vector{Main.DWI.Id}:
 Main.DWI.Workflow.Id()
 Main.DWI.Steps.Id{1}()
 Main.DWI.Steps.Id{2}()
 Main.DWI.Steps.Id{3}()
 Main.DWI.Steps.Id{4}()
 Main.DWI.Steps.Id{5}()
 Main.DWI.Steps.Id{6}()

In [10]:
wf = fnd(DWI.Workflow.Id(),DWI)

Main.DWI.Workflow.WF()

In [11]:
ps = pts(wf)

6-element Vector{Main.DWI.Steps.Id}:
 Main.DWI.Steps.Id{1}()
 Main.DWI.Steps.Id{2}()
 Main.DWI.Steps.Id{3}()
 Main.DWI.Steps.Id{4}()
 Main.DWI.Steps.Id{5}()
 Main.DWI.Steps.Id{6}()

In [12]:
CHAKRA.sequence([ps...,DWI.Steps.Id{10}()],DWI)

Main.CHAKRA.None()

In [13]:
s = fnd(ps[3],DWI)

Main.DWI.Steps.Step{3}()

In [14]:
getp(:STEP_NAME,s)

"Spring 2"

<a id='events'></a>
## Events

In [16]:
module Events

using Main.CHAKRA
using CSV
using DataFrames

df = DataFrame(CSV.File("STEPS.csv"))
idx = df[:,1]

# Abstract Types

struct Id{N} <: CHAKRA.Id end
struct Event{N} <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

# Attribute Definitions

abstract type Attribute{N,T} <: CHAKRA.Attribute{N,T} end

struct TASK_NAME <: Attribute{:TASK_NAME,String} end
CHAKRA.__attributes__(::Val{:TASK_NAME}) = TASK_NAME()

struct ORIGIN_NAME <: Attribute{:ORIGIN_NAME,String} end
CHAKRA.__attributes__(::Val{:ORIGIN_NAME}) = ORIGIN_NAME()

struct START_TIME <: Attribute{:START_TIME,String} end
CHAKRA.__attributes__(::Val{:START_TIME}) = START_TIME()

struct STEP_DURATION <: Attribute{:STEP_DURATION,Float64} end
CHAKRA.__attributes__(::Val{:STEP_DURATION}) = STEP_DURATION()

struct STEP_INDEX <: Attribute{:STEP_INDEX,Int} end
CHAKRA.__attributes__(::Val{:STEP_INDEX}) = STEP_INDEX()

struct STEP_RANK <: Attribute{:STEP_RANK,Int} end
CHAKRA.__attributes__(::Val{:STEP_RANK}) = STEP_RANK()

struct USER_NAME <: Attribute{:USER_NAME,String} end
CHAKRA.__attributes__(::Val{:USER_NAME}) = USER_NAME()

struct EXPERIENCE_LEVEL <: Attribute{:EXPERIENCE_LEVEL,Int} end
CHAKRA.__attributes__(::Val{:EXPERIENCE_LEVEL}) = EXPERIENCE_LEVEL()

# CHAKRA Interface

CHAKRA.fnd(::Id{N},::Data) where N = Event{N}()
CHAKRA.geta(::A,::Event{n}) where {n,N,A<:Attribute{N}} = df[n,:][N]
CHAKRA.getp(::TYPE,::Event) = "Arkite Event"
CHAKRA.pts(c::Event) = CHAKRA.Id[]
CHAKRA.dom(::Data) = [Id{i}() for i in idx]

__data__ = Data()

end

using Main.Events
dom(Events)

40-element Vector{Main.Events.Id}:
 Main.Events.Id{1}()
 Main.Events.Id{2}()
 Main.Events.Id{3}()
 Main.Events.Id{4}()
 Main.Events.Id{5}()
 Main.Events.Id{6}()
 Main.Events.Id{7}()
 Main.Events.Id{8}()
 Main.Events.Id{9}()
 Main.Events.Id{10}()
 Main.Events.Id{11}()
 Main.Events.Id{12}()
 Main.Events.Id{13}()
 ⋮
 Main.Events.Id{29}()
 Main.Events.Id{30}()
 Main.Events.Id{31}()
 Main.Events.Id{32}()
 Main.Events.Id{33}()
 Main.Events.Id{34}()
 Main.Events.Id{35}()
 Main.Events.Id{36}()
 Main.Events.Id{37}()
 Main.Events.Id{38}()
 Main.Events.Id{39}()
 Main.Events.Id{40}()

In [17]:
e = fnd(Events.Id{1}(),Events)

Main.Events.Event{1}()

In [18]:
pts(e)

Main.CHAKRA.Id[]

In [19]:
getp(:TYPE,e)

"Arkite Event"

In [20]:
geta(:START_TIME,e)

"26/04/2023 16:51:33"

<a id='Seqs'></a>
## Seqs

In [22]:
module Seqs

using Main.CHAKRA
using Main.Events

seqs = [1:4...]
seqparts(T) = [Events.Id{S}() for S in T*10-9:T*10]

struct Id{N} <: CHAKRA.Id end
struct Seq{N} <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

struct SEQ_TYPE <: CHAKRA.Property{:SEQ_TYPE,String} end
CHAKRA.__properties__(::Val{:SEQ_TYPE}) = SEQ_TYPE()

CHAKRA.fnd(::Id{N},::Data) where N = Seq{N}()
CHAKRA.pts(::Seq{N}) where N = seqparts(N)
CHAKRA.getp(::TYPE,::Seq) = "Arkite Sequence"
CHAKRA.getp(::SEQ_TYPE,::Seq) = "Successful Assembly"
CHAKRA.dom(::Data) = [Id{N}() for N in seqs]

__data__ = Data()

end

Main.Seqs

<a id='chunks'></a>
## Chunks

In [24]:
module Chunks

using Main.CHAKRA
using Main.Events

chunks = [1:20...]
chunkparts(I) = [Events.Id{I*2-1}(),Events.Id{I*2}()]

struct Id{I} <: CHAKRA.Id end
struct Chunk{I} <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(::Id{I},::Data) where I = Chunk{I}()
CHAKRA.pts(::Chunk{I}) where I = chunkparts(I)
CHAKRA.getp(::TYPE,::Chunk) = "Arkite Chunk"
CHAKRA.dom(::Data) = [Id{N}() for N in chunks]

__data__ = Data()

end

Main.Chunks

<a id='ChunkSeqs'></a>
## ChunkSeqs

In [26]:
module ChunkSeqs

using Main.CHAKRA
using Main.Chunks

chunkseqs = [1:4...]
parts(N) = [Chunks.Id{N*5-i}() for i in reverse([0:4...])] 

struct Id{N} <: CHAKRA.Id end
struct ChunkSeq{N} <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(::Id{N},::Data) where N = ChunkSeq{N}()
CHAKRA.pts(::ChunkSeq{N}) where N = parts(N)
CHAKRA.getp(::TYPE,::ChunkSeq) = "Chunk Sequence"
CHAKRA.dom(::Data) = [Id{N}() for N in chunkseqs]

__data__ = Data()

end

Main.ChunkSeqs

<a id='SeqEx'></a>
## SeqEx

In [28]:
module SeqEx

using Main.CHAKRA
using Main.Seqs
using Main.DWI

wf = DWI.Workflow.Id()

execs = [1:4...]
ps(N) = CHAKRA.Id[Seqs.Id{N}(),wf]

struct Id{N} <: CHAKRA.Id end
struct Exec{N} <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(::Id{N},::Data) where N = Exec{N}()
CHAKRA.pts(::Exec{N}) where N = ps(N)
CHAKRA.getp(::TYPE,::Exec) = "isExecutionOf"
CHAKRA.dom(::Data) = [Id{N}() for N in execs]

__data__ = Data()

end

Main.SeqEx

<a id='ChunkEx'></a>
## ChunkEx

In [30]:
module ChunkEx

using Main.CHAKRA
using Main.Chunks
using Main.DWI

execs = [1:16...]
steps = [DWI.Steps.Id{N}() for N in 1:6]
arcs = Dict([0=>6,1=>1,2=>4,3=>5])
ps(N) = CHAKRA.Id[Chunks.Id{N}(),steps[arcs[N%4]]]

struct Id{N} <: CHAKRA.Id end
struct Exec{N} <: CHAKRA.Constituent end
struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(::Id{N},::Data) where N = Exec{N}()
CHAKRA.pts(::Exec{N}) where N = ps(N)
CHAKRA.getp(::TYPE,::Exec) = "isExecutionOf"
CHAKRA.dom(::Data) = [Id{N}() for N in execs]

__data__ = Data()

end

Main.ChunkEx

<a id='KB'></a>
## KB

In [32]:
module KB

using Main.CHAKRA
using Main.Events
using Main.Seqs
using Main.Chunks
using Main.ChunkSeqs
using Main.DWI
using Main.SeqEx
using Main.ChunkEx

struct Data <: CHAKRA.Hierarchy end

CHAKRA.fnd(x::ID,::Data) where {ID<:CHAKRA.Id} = fnd(x,parentmodule(ID))
CHAKRA.dom(::Data) = [dom(ChunkEx)...,dom(SeqEx)...,dom(ChunkSeqs)...,dom(Chunks)...,dom(Seqs)...,dom(Events)...,dom(DWI)...]

__data__ = Data()

end

Main.KB

In [33]:
using .KB
dom(KB)
fnd(Main.ChunkSeqs.Id{1}(),KB)

Main.ChunkSeqs.ChunkSeq{1}()

<a id='query-language'></a>
## Query Language

In [35]:
PAIR(X,Y) = Pair{x,y} where {x<:X,y<:Y}
LST(X) = Vector{x} where {x<:X}
OPT(X) = Union{X,CHAKRA.None}
ID = CHAKRA.Id
C = CHAKRA.Constituent
CON = PAIR(ID,C)
G = LST(CON)

domain(cs::G) = first.(cs)

mem(x::ID,cs::G)::Bool = x in domain(cs)
mem(xs::LST(ID),cs::G) = all([mem(x,cs) for x in xs])

position(x::ID,cs::G) = findfirst(==(x),domain(cs))

lookup(x::ID,cs::G) = mem(x,cs) ? cs[position(x,cs)] : none

lookup(xs::LST(ID),cs::G) = begin
    res = Pair{CHAKRA.Id,CHAKRA.Constituent}[]
    for x in xs
        c = lookup(x,cs)
        c == none && return none
        union!(res,[c])
    end
    return res
end

pop(x::ID,cs::G) = begin 
    !mem(x,cs) && return CON[]
    return cs[position(x,cs):end]
end

parts(x::ID,cs::G) = begin 
    !mem(x,cs) && return none
    c = lookup(x,cs)
    return lookup(pts(c[2]),cs)
end
parts(c::CON,cs::G) = lookup(pts(c[2]),cs)

subparts(x::ID,cs::G) = begin
    isempty(cs) && return CON[]
    c = lookup(x,cs)
    c == none && return CON[]
    res = CON[c]
    ps = pts(c[2])
    for p in ps
        union!(res,subparts(p,pop(p,cs)))
    end
    return res
end

subparts(xs::LST(ID),cs::G) = begin
    res = CON[]
    [union!(res,subparts(x,cs)) for x in xs]
    return [c for c in cs if c in res]
end

superparts(x::ID,cs::G) = begin
    isempty(cs) && return CON[]
    c = lookup(x,cs)
    c == none && return CON[]
    res = CON[c]
    for c2 in reverse(cs[1:position(x,cs)])
        if x in pts(c2[2])
            push!(res,c2)
            union!(res,superparts(c2[1],cs))
        end
    end
    return reverse(res)
end

superparts(xs::LST(ID),cs::G) = begin 
    res = CON[]
    [union!(res,superparts(x,cs)) for x in xs]
    return [c for c in cs if c in res]
end

haspart(x::ID,y::ID,cs::G) = begin
    !(mem(x,cs)) && return false
    return y in first.(parts(x,cs))
end

ispartof(x::ID,y::ID,cs::G) = haspart(y,x,cs)

hassubpart(x,y,cs) = begin
    y in first.(subparts(x,cs))
end

issubpartof(x::ID,y::ID,cs::G) = hassubpart(y,x,cs)

isdag(cs::G) = begin 
    isempty(cs) && return true
    x,c = cs[1]
    t = cs[2:end]
    ps = pts(c)
    mem(x,t) && (print("!!!"); return false)
    !mem(ps,t) && (print("!!!$x"); return false)
    !isdag(t) && (print("!!!"); return false)
    return true
end
isleaf(x::ID,cs::G) = begin 
    c = lookup(x,cs) 
    return c != none && isempty(pts(c[2]))
end
istree(cs::G) = begin
    isempty(cs) && return true
    x,c = cs[1]
    t = cs[2:end]
    sps = [subparts(p[1],cs) for p in parts(x,cs)]
    for p in parts(x)
        return isleaf(p,cs) || istree(p,cs)
    end
end

istree (generic function with 1 method)

In [36]:
wf = DWI.Workflow.Id()::ID
xs = dom(KB);
cs = cts(KB);
x = xs[1]

Main.ChunkEx.Id{1}()

In [37]:
cs

95-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
   Main.ChunkEx.Id{1}() => Main.ChunkEx.Exec{1}()
   Main.ChunkEx.Id{2}() => Main.ChunkEx.Exec{2}()
   Main.ChunkEx.Id{3}() => Main.ChunkEx.Exec{3}()
   Main.ChunkEx.Id{4}() => Main.ChunkEx.Exec{4}()
   Main.ChunkEx.Id{5}() => Main.ChunkEx.Exec{5}()
   Main.ChunkEx.Id{6}() => Main.ChunkEx.Exec{6}()
   Main.ChunkEx.Id{7}() => Main.ChunkEx.Exec{7}()
   Main.ChunkEx.Id{8}() => Main.ChunkEx.Exec{8}()
   Main.ChunkEx.Id{9}() => Main.ChunkEx.Exec{9}()
  Main.ChunkEx.Id{10}() => Main.ChunkEx.Exec{10}()
  Main.ChunkEx.Id{11}() => Main.ChunkEx.Exec{11}()
  Main.ChunkEx.Id{12}() => Main.ChunkEx.Exec{12}()
  Main.ChunkEx.Id{13}() => Main.ChunkEx.Exec{13}()
                        ⋮
   Main.Events.Id{36}() => Main.Events.Event{36}()
   Main.Events.Id{37}() => Main.Events.Event{37}()
   Main.Events.Id{38}() => Main.Events.Event{38}()
   Main.Events.Id{39}() => Main.Events.Event{39}()
   Main.Events.Id{40}() => Main.Events.Event{40}()

In [38]:
position(xs[10],cs)

10

In [39]:
mem(x,cs)

true

In [40]:
lookup(x,cs)

Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.ChunkEx.Id{1}(), Main.ChunkEx.Exec{1}())

In [41]:
lookup(xs[1:20],cs)

20-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
  Main.ChunkEx.Id{1}() => Main.ChunkEx.Exec{1}()
  Main.ChunkEx.Id{2}() => Main.ChunkEx.Exec{2}()
  Main.ChunkEx.Id{3}() => Main.ChunkEx.Exec{3}()
  Main.ChunkEx.Id{4}() => Main.ChunkEx.Exec{4}()
  Main.ChunkEx.Id{5}() => Main.ChunkEx.Exec{5}()
  Main.ChunkEx.Id{6}() => Main.ChunkEx.Exec{6}()
  Main.ChunkEx.Id{7}() => Main.ChunkEx.Exec{7}()
  Main.ChunkEx.Id{8}() => Main.ChunkEx.Exec{8}()
  Main.ChunkEx.Id{9}() => Main.ChunkEx.Exec{9}()
 Main.ChunkEx.Id{10}() => Main.ChunkEx.Exec{10}()
 Main.ChunkEx.Id{11}() => Main.ChunkEx.Exec{11}()
 Main.ChunkEx.Id{12}() => Main.ChunkEx.Exec{12}()
 Main.ChunkEx.Id{13}() => Main.ChunkEx.Exec{13}()
 Main.ChunkEx.Id{14}() => Main.ChunkEx.Exec{14}()
 Main.ChunkEx.Id{15}() => Main.ChunkEx.Exec{15}()
 Main.ChunkEx.Id{16}() => Main.ChunkEx.Exec{16}()
    Main.SeqEx.Id{1}() => Main.SeqEx.Exec{1}()
    Main.SeqEx.Id{2}() => Main.SeqEx.Exec{2}()
    Main.SeqEx.Id{3}() => Main.SeqEx.Exec{3}()
  

In [42]:
pop(xs[50],cs)

46-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
    Main.Events.Id{2}() => Main.Events.Event{2}()
    Main.Events.Id{3}() => Main.Events.Event{3}()
    Main.Events.Id{4}() => Main.Events.Event{4}()
    Main.Events.Id{5}() => Main.Events.Event{5}()
    Main.Events.Id{6}() => Main.Events.Event{6}()
    Main.Events.Id{7}() => Main.Events.Event{7}()
    Main.Events.Id{8}() => Main.Events.Event{8}()
    Main.Events.Id{9}() => Main.Events.Event{9}()
   Main.Events.Id{10}() => Main.Events.Event{10}()
   Main.Events.Id{11}() => Main.Events.Event{11}()
   Main.Events.Id{12}() => Main.Events.Event{12}()
   Main.Events.Id{13}() => Main.Events.Event{13}()
   Main.Events.Id{14}() => Main.Events.Event{14}()
                        ⋮
   Main.Events.Id{36}() => Main.Events.Event{36}()
   Main.Events.Id{37}() => Main.Events.Event{37}()
   Main.Events.Id{38}() => Main.Events.Event{38}()
   Main.Events.Id{39}() => Main.Events.Event{39}()
   Main.Events.Id{40}() => Main.Events.Event{40}(

In [43]:
xs[1]
pts(lookup(xs[1],cs)[2])

2-element Vector{Main.CHAKRA.Id}:
 Main.Chunks.Id{1}()
 Main.DWI.Steps.Id{1}()

In [44]:
parts(xs[1],cs)

2-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
    Main.Chunks.Id{1}() => Main.Chunks.Chunk{1}()
 Main.DWI.Steps.Id{1}() => Main.DWI.Steps.Step{1}()

In [45]:
subparts(x,cs)

5-element Vector{Pair{x, y} where {x<:Main.CHAKRA.Id, y<:Main.CHAKRA.Constituent}}:
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.ChunkEx.Id{1}(), Main.ChunkEx.Exec{1}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Chunks.Id{1}(), Main.Chunks.Chunk{1}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{1}(), Main.Events.Event{1}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{2}(), Main.Events.Event{2}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.DWI.Steps.Id{1}(), Main.DWI.Steps.Step{1}())

In [46]:
subparts(xs[1:2],cs)

10-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
   Main.ChunkEx.Id{1}() => Main.ChunkEx.Exec{1}()
   Main.ChunkEx.Id{2}() => Main.ChunkEx.Exec{2}()
    Main.Chunks.Id{1}() => Main.Chunks.Chunk{1}()
    Main.Chunks.Id{2}() => Main.Chunks.Chunk{2}()
    Main.Events.Id{1}() => Main.Events.Event{1}()
    Main.Events.Id{2}() => Main.Events.Event{2}()
    Main.Events.Id{3}() => Main.Events.Event{3}()
    Main.Events.Id{4}() => Main.Events.Event{4}()
 Main.DWI.Steps.Id{1}() => Main.DWI.Steps.Step{1}()
 Main.DWI.Steps.Id{4}() => Main.DWI.Steps.Step{4}()

In [47]:
superparts(DWI.Steps.Id{6}(),cs)

10-element Vector{Pair{x, y} where {x<:Main.CHAKRA.Id, y<:Main.CHAKRA.Constituent}}:
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.ChunkEx.Id{4}(), Main.ChunkEx.Exec{4}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.ChunkEx.Id{8}(), Main.ChunkEx.Exec{8}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.ChunkEx.Id{12}(), Main.ChunkEx.Exec{12}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.ChunkEx.Id{16}(), Main.ChunkEx.Exec{16}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.SeqEx.Id{4}(), Main.SeqEx.Exec{4}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.SeqEx.Id{3}(), Main.SeqEx.Exec{3}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.SeqEx.Id{2}(), Main.SeqEx.Exec{2}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.SeqEx.Id{1}(), Main.SeqEx.Exec{1}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.DWI.Workflow.Id(), Main.DWI.Workflow.WF())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.DWI.Steps.Id{6}(), Main.DWI.Steps.St

In [48]:
mem(SeqEx.Id{1}(),cs)

true

In [49]:
haspart(ChunkSeqs.Id{1}(),Chunks.Id{1}(),cs)

true

In [50]:
haspart(DWI.Workflow.Id(),DWI.Steps.Id{1}(),cs)

true

In [51]:
haspart(SeqEx.Id{1}(),Events.Id{5}(),cs)

false

In [52]:
hassubpart(SeqEx.Id{1}(),Events.Id{5}(),cs)

true

In [53]:
hassubpart(SeqEx.Id{1}(),Events.Id{15}(),cs)

false

In [54]:
isdag(cs)

true

In [55]:
PTY{N,T} = CHAKRA.Property{N,t} where {t<:T}
ATT{N,T} = CHAKRA.Attribute{N,t} where {t<:T}

# READ(A) : G -> option A x G 

abstract type READ{A} end

(R::READ{A})(cs::G) where A = R.op(cs)::option{A}

struct Read{A} <: READ{A}
    op::Function
    Read(A,f) = new{A}(f)
    Read(A) = new{A}(cs->none)
end

abstract type KLEISLI{A,B} end

function (K::KLEISLI{A,B})(a) where {A,B}
    a==none && return Read(B)
    Read(B,K.op(a))
end

function(K::KLEISLI{A,B})(a::A,cs::G) where {A,B}
    K(a)(cs)
end

struct BIND{A,B} <: READ{B}
    op
    BIND(R::READ{A},F::KLEISLI{A,B}) where {A,B} = new{A,B}(cs->F(R(cs),cs))
end

struct FISH{A,B} <: KLEISLI{A,B}
    op
    FISH(F::KLEISLI{A,B},G::KLEISLI{B,C}) where {A,B,C} = new{A,C}(x->cs->G(F(x,cs),cs))
end

FISH(f::KLEISLI,gs::KLEISLI...) = isempty(gs) ? f : FISH(f,FISH(gs...))

QUERY(R::READ,KS::KLEISLI...) = BIND(R,FISH(KS...))

struct PTS <: KLEISLI{CON,LST(CON)}
    op
    PTS() = new(c->cs->parts(c,cs))
end

struct GETA{N,T} <: KLEISLI{CON,T}
    op
    GETA(a::ATT{N,T}) where {N,T} = new{N,T}(c->cs->geta(a,c[2]))
    GETA(N::Symbol) = GETA(CHAKRA.__attributes__(N))
end

struct GETP{N,T} <: KLEISLI{CON,T}
    op
    GETP(a::PTY{N,T}) where {N,T} = new{N,T}(c->cs->getp(a,c[2]))
    GETP(N::Symbol) = GETP(CHAKRA.__properties__(N))
end

struct HASA <: KLEISLI{CON,Bool}
    op
    HASA(N::Symbol,v) = new(c->cs->geta(N,c[2])==v)
    HASA(N::Symbol) = new(c->cs->geta(N,c[2])!=none)
end

struct HASP <: KLEISLI{CON,Bool}
    op
    HASP(N::Symbol,v) = new(c->cs->getp(N,c[2])==v)
    HASP(N::Symbol) = new(c->cs->getp(N,c[2])!=none)
end

struct FIRSTPART <: KLEISLI{CON,Bool}
    op
    FIRSTPART(T::KLEISLI) = new(c->cs->(ps = parts(c,cs); isempty(ps) ? false : T(first(ps))(cs)))
    FIRSTPART(x::ID) = FIRSTPART(c->cs->c[1]==x)
end

struct LASTPART <: KLEISLI{CON,Bool}
    op
    LASTPART(T) = new(c->cs->(ps = parts(c,cs); isempty(ps) ? false : T(last(ps))(cs)))
    LASTPART(x::ID) = LASTPART(c->cs->c[1]==x)
end

struct ALLPARTS <: KLEISLI{CON,Bool}
    op
    ALLPARTS(T) = new(c->cs->all([T(p,cs) for p in parts(c,cs)])) 
end

struct SOMEPART <: KLEISLI{CON,Bool}
    op
    SOMEPART(T) = new(c->cs->any([T(p,cs) for p in parts(c,cs)]))
end

struct AND <: KLEISLI{CON,Bool}
    op
    AND(f::KLEISLI,g::KLEISLI) = new(a->cs->f(a,cs)&&g(a,cs))
    AND(f::KLEISLI,gs::KLEISLI...) = isempty(gs) ? f : AND(AND(f,gs[1]),gs[2:end]...)
end

struct SELECT <: READ{G}
    op
    SELECT(T::KLEISLI{CON,Bool}) = new(cs->[c for c in cs if T(c)(cs)])
end

opmap(B,f,l) = begin
    isempty(l) && return B[]
    h = f(l[1])
    t = opmap(B,f,l[2:end])
    h == none && return none
    t == none && return none
    return B[h,t...]
end

struct MAP{A,B} <: KLEISLI{LST(A),LST(B)}
    op
    MAP(f::KLEISLI{A,B}) where {A,B} = new{A,B}(l->cs->opmap(B,x->f(x)(cs),l))
end

struct RET{A} <: READ{A}
    op
    RET(v::A) where A = new{A}(_->v)
end

struct SUBCS <: KLEISLI{CON,G}
    op
    SUBCS() = new(c->cs->subparts(c,cs))
end

struct PROJ <: KLEISLI{CON,G}
    op
    PROJ(T::KLEISLI{CON,Bool}) = new(c->cs->CON[x for x in subparts(c[1],cs) if T(x,cs)])
end

struct SUM <: KLEISLI{LST(Float64),Float64}
    op
    SUM() = new(l->cs->sum(l))
end

struct AVG <: KLEISLI{LST(Float64),Float64}
    op
    AVG() = new(l->cs->sum(l)/length(l))
end

<a id='Querying'></a>
## Querying

In [57]:
P = HASP(:TYPE,"isExecutionOf")
@time cs |> SELECT(P)

  0.250917 seconds (167.21 k allocations: 11.305 MiB, 2.15% gc time, 99.57% compilation time)


20-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
  Main.ChunkEx.Id{1}() => Main.ChunkEx.Exec{1}()
  Main.ChunkEx.Id{2}() => Main.ChunkEx.Exec{2}()
  Main.ChunkEx.Id{3}() => Main.ChunkEx.Exec{3}()
  Main.ChunkEx.Id{4}() => Main.ChunkEx.Exec{4}()
  Main.ChunkEx.Id{5}() => Main.ChunkEx.Exec{5}()
  Main.ChunkEx.Id{6}() => Main.ChunkEx.Exec{6}()
  Main.ChunkEx.Id{7}() => Main.ChunkEx.Exec{7}()
  Main.ChunkEx.Id{8}() => Main.ChunkEx.Exec{8}()
  Main.ChunkEx.Id{9}() => Main.ChunkEx.Exec{9}()
 Main.ChunkEx.Id{10}() => Main.ChunkEx.Exec{10}()
 Main.ChunkEx.Id{11}() => Main.ChunkEx.Exec{11}()
 Main.ChunkEx.Id{12}() => Main.ChunkEx.Exec{12}()
 Main.ChunkEx.Id{13}() => Main.ChunkEx.Exec{13}()
 Main.ChunkEx.Id{14}() => Main.ChunkEx.Exec{14}()
 Main.ChunkEx.Id{15}() => Main.ChunkEx.Exec{15}()
 Main.ChunkEx.Id{16}() => Main.ChunkEx.Exec{16}()
    Main.SeqEx.Id{1}() => Main.SeqEx.Exec{1}()
    Main.SeqEx.Id{2}() => Main.SeqEx.Exec{2}()
    Main.SeqEx.Id{3}() => Main.SeqEx.Exec{3}()
  

In [58]:
P = AND(HASP(:TYPE,"Arkite Sequence"),
        HASP(:SEQ_TYPE,"Successful Assembly"))
@time cs |> SELECT(FIRSTPART(P))

  0.631732 seconds (1.11 M allocations: 71.967 MiB, 0.69% gc time, 99.40% compilation time)


4-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
 Main.SeqEx.Id{1}() => Main.SeqEx.Exec{1}()
 Main.SeqEx.Id{2}() => Main.SeqEx.Exec{2}()
 Main.SeqEx.Id{3}() => Main.SeqEx.Exec{3}()
 Main.SeqEx.Id{4}() => Main.SeqEx.Exec{4}()

In [59]:
P = LASTPART(wf)
@time cs |> SELECT(P)

  0.040562 seconds (34.92 k allocations: 2.624 MiB, 96.66% compilation time)


4-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
 Main.SeqEx.Id{1}() => Main.SeqEx.Exec{1}()
 Main.SeqEx.Id{2}() => Main.SeqEx.Exec{2}()
 Main.SeqEx.Id{3}() => Main.SeqEx.Exec{3}()
 Main.SeqEx.Id{4}() => Main.SeqEx.Exec{4}()

In [60]:
S = SELECT(AND(HASP(:TYPE,"isExecutionOf"),
               FIRSTPART(AND(HASP(:TYPE,"Arkite Sequence"),
                             HASP(:SEQ_TYPE,"Successful Assembly"))),
               LASTPART(wf)))
@time es = cs |> S

  0.030498 seconds (29.42 k allocations: 2.014 MiB, 97.74% compilation time)


4-element Vector{Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}}:
 Main.SeqEx.Id{1}() => Main.SeqEx.Exec{1}()
 Main.SeqEx.Id{2}() => Main.SeqEx.Exec{2}()
 Main.SeqEx.Id{3}() => Main.SeqEx.Exec{3}()
 Main.SeqEx.Id{4}() => Main.SeqEx.Exec{4}()

In [61]:
@time s = cs |> PROJ(HASP(:TYPE,"Arkite Event"))(es[1])

  0.030791 seconds (39.30 k allocations: 2.631 MiB, 97.82% compilation time)


10-element Vector{Pair{x, y} where {x<:Main.CHAKRA.Id, y<:Main.CHAKRA.Constituent}}:
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{1}(), Main.Events.Event{1}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{2}(), Main.Events.Event{2}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{3}(), Main.Events.Event{3}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{4}(), Main.Events.Event{4}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{5}(), Main.Events.Event{5}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{6}(), Main.Events.Event{6}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{7}(), Main.Events.Event{7}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{8}(), Main.Events.Event{8}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{9}(), Main.Events.Event{9}())
 Pair{Main.CHAKRA.Id, Main.CHAKRA.Constituent}(Main.Events.Id{10}(), Main.Events.Event{

In [62]:
@time ds = cs |> MAP(GETA(:STEP_DURATION))(s)

  0.072385 seconds (170.25 k allocations: 11.661 MiB, 6.73% gc time, 99.29% compilation time)


10-element Vector{Float64}:
 13338.2882
     0.0
  6809.3036
 10938.0163
  7161.9941
  4097.3993
 24735.4684
  4426.8248
  4034.7147
     0.0

In [63]:
@time cs |> SUM()(ds)

  0.010610 seconds (3.68 k allocations: 257.609 KiB, 99.52% compilation time)


75542.0094

In [64]:
@time cs |> QUERY(SELECT(AND(HASP(:TYPE,"isExecutionOf"),
                       FIRSTPART(AND(HASP(:TYPE,"Arkite Sequence"),
                                     HASP(:SEQ_TYPE,"Successful Assembly"))),
                       LASTPART(DWI.Workflow.Id()))),
            MAP(FISH(PROJ(HASP(:TYPE,"Arkite Event")),MAP(GETA(:STEP_DURATION)),SUM())),
            AVG())

  0.289939 seconds (172.19 k allocations: 11.132 MiB, 98.52% compilation time)


39489.89967499999

In [65]:
@time cs |> QUERY(SELECT(AND(HASP(:TYPE,"isExecutionOf"),
                             FIRSTPART(HASP(:TYPE,"Arkite Chunk")),
                             LASTPART(DWI.Steps.Id{6}()))),
                  MAP(FISH(PROJ(HASP(:TYPE,"Arkite Event")),MAP(GETA(:STEP_DURATION)),SUM())),AVG())

  0.050666 seconds (32.93 k allocations: 2.255 MiB, 96.14% compilation time)


13625.193049999998

<a id='json-queries'></a>
## QUERY in JSON Format

```json
{
    "op": "QUERY",
    "args": [
	{"op": "SELECT",
	 "args": [
	     {"op": "AND",
	      "args": [
		  {"op": "HASP",
		   "args": [
		       ":TYPE",
		       "\"isExecutionOf\""]},
		  {"op": "LASTPART",
		   "args": [
		       "DWI.Workflow.Id()"]}]}]},
	{"op":"MAP",
	 "args": [
	     {"op":"FISH",
	      "args": [
		  {"op":"PROJ",
		   "args": [
		       {"op": "HASP",
			"args": [
			    ":TYPE",
			    "\"Arkite Event\""]}]},
		  {"op":"MAP",
		   "args": [
		       {"op": "GETA",
			"args": [
			    ":STEP_DURATION"]}]},
		  {"op":"SUM",
		   "args": []}]}]},
	{"op": "AVG",
	 "args": []}]
}
```

In [68]:
function chparse(x) 
    if x isa Dict
        op = x["op"]
        args = x["args"]
        return eval(Symbol(op))([chparse(a) for a in args]...)
    end
    return eval(Meta.parse(x))
end;

In [69]:
import JSON
d = JSON.parsefile("query.json");

In [70]:
x = chparse(d);

In [71]:
cs |> x

39489.89967499999

In [72]:
function EXECUTE(d::Dict,kb)
    cts(kb) |> chparse(d)
end
function EXECUTE(q::String,kb)
    d = JSON.parsefile(q)
    return EXECUTE(d,kb)
end

EXECUTE (generic function with 2 methods)

In [73]:
EXECUTE(d,KB)

39489.89967499999

## Querying Knowledge Modules from Python

```
source $HOME/.venv/bin/activate
python3
```

```python
import julia
#julia.install()
j = julia.Julia()
x = j.include("demo-v2.jl")
j.eval("EXECUTE(\"query.json\",KB)")
```